In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
con = pd.read_csv('/content/drive/MyDrive/LG Hellovision VOD 추천 프로젝트/모델/content_surprise/csv/click.csv', encoding='cp949')
con.columns = ['subsr', 'program', 'vod_id', 'main_cat', 'sub_cat', 'category', 'click_score']
con

,subsr,program,vod_id,main_cat,sub_cat,category,click_score
0,65968000,핑크퐁 동물 동요,882,키즈,기타,키즈,1.000000
1,63086000,지니강이 플러스 시즌7,1622,키즈,기타,키즈,0.012195
2,63086000,지니강이 플러스 시즌9,2043,키즈,기타,키즈,0.006098
3,62733000,엘리가간다 시즌12,1958,키즈,기타,키즈,0.500000
4,63086000,지니강이 플러스 시즌8,1596,키즈,기타,키즈,0.012195
...,...,...,...,...,...,...,...
7352,64982000,포켓몬스터 XY and Z,2918,키즈,애니메이션,키즈,0.014493
7353,66864000,포켓몬스터 썬 앤 문 3,3135,키즈,애니메이션,키즈,0.045455
7354,59917000,라바 인 뉴욕,10,키즈,오락,키즈,0.500000
7355,63086000,사이틴 시즌1,3468,키즈,학습,키즈,0.006098


In [53]:
vod_list = pd.read_csv('/content/drive/MyDrive/LG Hellovision VOD 추천 프로젝트/모델/content_surprise/csv/vod_list_add10.csv', index_col=0)
vod_id = vod_list[['rename', 'Category', 'vod_id']]
vod_id.columns = ['rename', 'category', 'vod_id']
vod_id

,rename,category,vod_id
0,와이낫크루뜻밖의여행,TV프로그램,0
1,그레이트뷰티,영화,1
2,해안선,영화,2
3,아는형님,TV프로그램,3
4,베놈,영화,4
...,...,...,...
5019,일루셔니스트,영화,5019
5020,"파리,13구",영화,5020
5021,미쓰백,영화,5021
5022,그녀는거짓말을너무사랑해,영화,5022


In [24]:
con1 = con.copy()
con1

,subsr,program,vod_id,main_cat,sub_cat,category,click_score
0,65968000,핑크퐁 동물 동요,882,키즈,기타,키즈,1.000000
1,63086000,지니강이 플러스 시즌7,1622,키즈,기타,키즈,0.012195
2,63086000,지니강이 플러스 시즌9,2043,키즈,기타,키즈,0.006098
3,62733000,엘리가간다 시즌12,1958,키즈,기타,키즈,0.500000
4,63086000,지니강이 플러스 시즌8,1596,키즈,기타,키즈,0.012195
...,...,...,...,...,...,...,...
7352,64982000,포켓몬스터 XY and Z,2918,키즈,애니메이션,키즈,0.014493
7353,66864000,포켓몬스터 썬 앤 문 3,3135,키즈,애니메이션,키즈,0.045455
7354,59917000,라바 인 뉴욕,10,키즈,오락,키즈,0.500000
7355,63086000,사이틴 시즌1,3468,키즈,학습,키즈,0.006098


In [25]:
con_score = con1[['subsr', 'vod_id', 'click_score']]
con_score

,subsr,vod_id,click_score
0,65968000,882,1.000000
1,63086000,1622,0.012195
2,63086000,2043,0.006098
3,62733000,1958,0.500000
4,63086000,1596,0.012195
...,...,...,...
7352,64982000,2918,0.014493
7353,66864000,3135,0.045455
7354,59917000,10,0.500000
7355,63086000,3468,0.006098


In [ ]:
! pip install surprise

In [26]:
# 필요한 Surprise 알고리즘 불러오기
from surprise import SVD, BaselineOnly, SVDpp, Reader
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import KNNBaseline
from surprise import accuracy
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

# surprise 데이터 형식으로 변환
def convert_traintest_dataframe_forsurprise(training_dataframe):
    reader = Reader(rating_scale=(0, 1)) # 범위가 0~1 인 경우
    trainset = Dataset.load_from_df(training_dataframe[['subsr', 'vod_id', 'click_score']], reader)
    trainset = trainset.construct_trainset(trainset.raw_ratings)
    return trainset

trainset = convert_traintest_dataframe_forsurprise(con_score)

### KNNBaseline - user-based similarity

In [17]:
sim_options = {'name': 'cosine',
               'user_based': True  # user-based similarity(사용자 기반 협업 필터)
               }
# 최근접 이웃의 수를 6으로 지정, 훈련할 때 사용되는 난수 발생 시드 지정
knnbaseline = KNNBaseline(k = 6, sim_options=sim_options, random_state = 42)
knnbaseline.fit(trainset)

Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.


In [33]:
user_id = sorted(con_score.subsr.unique())
vod_id = sorted(con_score.vod_id.unique())

result = []
for user in user_id:
    for vod in vod_id:
        result.append(knnbaseline.predict(user, vod)[0:4])

result = pd.DataFrame(result, columns = ['subsr', 'vod_id', 'real', 'predict'])
# result = result[['subsr', 'vod_id', 'predict']]
result

,subsr,vod_id,real,predict
0,59879000,0,None,0.129002
1,59879000,1,None,0.129262
2,59879000,3,None,0.184380
3,59879000,5,None,0.127621
4,59879000,6,None,0.127250
...,...,...,...,...
2409315,67164000,4708,None,0.145030
2409316,67164000,4710,None,0.145343
2409317,67164000,4711,None,0.145322
2409318,67164000,4712,None,0.145528


In [36]:
result = result[['subsr', 'vod_id', 'predict']]
result

,subsr,vod_id,predict
0,59879000,0,0.129002
1,59879000,1,0.129262
2,59879000,3,0.184380
3,59879000,5,0.127621
4,59879000,6,0.127250
...,...,...,...
2409315,67164000,4708,0.145030
2409316,67164000,4710,0.145343
2409317,67164000,4711,0.145322
2409318,67164000,4712,0.145528


In [19]:
result_1 = []
for user in user_id:
    result_1.append(result[result['subsr'] == user].sort_values(by = 'predict', ascending=False).vod_id[:30].tolist())

result_1 = pd.DataFrame(result_1)
result_1 = pd.concat([pd.DataFrame(user_id, columns = ['subsr']), result_1], axis = 1)
result_1

,subsr,0,1,2,3,4,5,6,7,8,...,20,21,22,23,24,25,26,27,28,29
0,59879000,2865,296,1233,538,2867,2630,1877,581,240,...,1958,32,759,432,1391,1215,2024,289,335,3196
1,59882000,335,1401,1394,2226,754,143,610,336,991,...,847,112,2879,1191,2556,1958,757,788,1310,759
2,59886000,738,818,3891,610,448,3954,1877,36,847,...,2024,289,335,1599,3568,3183,3196,1227,1772,4223
3,59890000,8,296,2608,2147,538,2716,2867,1877,1233,...,1958,32,1215,1200,2024,289,1267,613,335,3705
4,59892000,1243,296,1166,240,2953,3892,139,3779,143,...,16,1394,3038,3327,976,315,852,849,317,316
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
894,67149000,3838,315,2235,2338,738,112,498,245,2592,...,143,2865,3385,507,1557,296,1200,4129,991,2028
895,67154000,1233,476,652,2866,2867,1877,581,240,780,...,888,32,432,1215,1200,2024,289,335,1227,3705
896,67160000,1243,2630,3838,241,143,2338,3892,4018,112,...,1074,738,1557,976,784,2633,1166,849,4129,991
897,67161000,335,448,2226,4018,991,538,726,3824,1166,...,2556,360,784,1958,32,1200,2024,289,2537,3141


In [50]:
vod_predict = result_1.copy()
vod_predict.index = user_id
vod_predict = vod_predict.apply(lambda x : x[1:].tolist(), axis = 1)
vod_predict = vod_predict.reset_index()
vod_predict.columns = ['subsr', 'vod_id']
vod_predict

,subsr,vod_id
0,59879000,"[2865, 296, 1233, 538, 2867, 2630, 1877, 581, ..."
1,59882000,"[335, 1401, 1394, 2226, 754, 143, 610, 336, 99..."
2,59886000,"[738, 818, 3891, 610, 448, 3954, 1877, 36, 847..."
3,59890000,"[8, 296, 2608, 2147, 538, 2716, 2867, 1877, 12..."
4,59892000,"[1243, 296, 1166, 240, 2953, 3892, 139, 3779, ..."
...,...,...
894,67149000,"[3838, 315, 2235, 2338, 738, 112, 498, 245, 25..."
895,67154000,"[1233, 476, 652, 2866, 2867, 1877, 581, 240, 7..."
896,67160000,"[1243, 2630, 3838, 241, 143, 2338, 3892, 4018,..."
897,67161000,"[335, 448, 2226, 4018, 991, 538, 726, 3824, 11..."


In [71]:
# 추천 VOD가 영화인 경우, 본 적이 있다면 추천안함
TV_kids = vod_list[(vod_list['Category'] == 'TV프로그램') | (vod_list['Category'] == '키즈')].vod_id.unique().tolist()
movie = vod_list[vod_list['Category'] == '영화'].vod_id.unique().tolist()

vod_predict_1 = pd.DataFrame()
for user in user_id:
    li = []
    watched_list = con_score[con_score.subsr == user].vod_id.tolist()
    rec_list = vod_predict[vod_predict.subsr == user].vod_id.tolist()[0]
    for x in rec_list:
        if x not in watched_list:
            li.append(x)
        elif x in watched_list and x in TV_kids:
            li.append(x)
        elif x in watched_list and x in movie:
            continue

    vod_predict_1 = pd.concat([vod_predict_1, pd.DataFrame(li).T], axis= 0)

vod_predict_1.index = user_id
vod_predict_1 = vod_predict_1.apply(lambda x : x.tolist(), axis = 1)
vod_predict_1 = vod_predict_1.reset_index()
vod_predict_1.columns = ['subsr', 'vod_id']
vod_predict_1

,subsr,vod_id
0,59879000,"[2865.0, 296.0, 1233.0, 538.0, 2867.0, 2630.0,..."
1,59882000,"[335.0, 1401.0, 1394.0, 2226.0, 754.0, 143.0, ..."
2,59886000,"[738.0, 818.0, 3891.0, 610.0, 448.0, 3954.0, 1..."
3,59890000,"[8.0, 296.0, 2608.0, 2147.0, 538.0, 2867.0, 18..."
4,59892000,"[1243.0, 296.0, 1166.0, 240.0, 2953.0, 3892.0,..."
...,...,...
894,67149000,"[3838.0, 315.0, 2235.0, 2338.0, 738.0, 112.0, ..."
895,67154000,"[1233.0, 476.0, 652.0, 2866.0, 2867.0, 1877.0,..."
896,67160000,"[1243.0, 3838.0, 241.0, 143.0, 2338.0, 3892.0,..."
897,67161000,"[335.0, 448.0, 2226.0, 4018.0, 991.0, 538.0, 7..."


In [82]:
testdata = pd.read_csv('/content/drive/MyDrive/LG Hellovision VOD 추천 프로젝트/모델/content_surprise/csv/watched_vod_10.csv', index_col=0)
testdata = testdata.groupby('subsr')['vod_id'].unique().reset_index()
testdata

,subsr,vod_id
0,59900000,"[1278, 1885]"
1,59930000,[2098]
2,59933000,[296]
3,60050000,"[200, 2415, 2869, 2546, 213, 2291, 2610, 4880]"
4,60067000,"[1785, 887, 1347, 4773, 4782, 4759, 992, 3806,..."
...,...,...
215,67133000,[296]
216,67164000,"[4956, 1434, 4995, 2058]"
217,67170000,"[3464, 948, 3900, 675]"
218,67202000,[1028]


In [83]:
# precision@k
def precision_k(testdata, recommended_data, K = 10):
    merge_df = pd.merge(testdata, recommended_data, on='subsr', how = 'left', suffixes=('_actual', '_rec'))
    merge_df = merge_df.dropna()
    precision_value = 0
    for i in range(merge_df.shape[0]):
        temp = len(set(merge_df.iloc[i,1]).intersection(set(merge_df.iloc[i, 2][:K]))) / K
        precision_value += temp

    return precision_value / merge_df.shape[0]

precision_k(testdata, vod_predict_1)

0.026153846153846153

### KNNBaseline - item-based similarity

In [92]:
sim_options = {'name': 'cosine',
               'user_based': False  # user-based similarity(사용자 기반 협업 필터)
               }
# 최근접 이웃의 수를 6으로 지정, 훈련할 때 사용되는 난수 발생 시드 지정
knnbaseline = KNNBaseline(k = 6, sim_options=sim_options, random_state = 42)
knnbaseline.fit(trainset)

Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.


In [93]:
user_id = sorted(con_score.subsr.unique())
vod_id = sorted(con_score.vod_id.unique())

result = []
for user in user_id:
    for vod in vod_id:
        result.append(knnbaseline.predict(user, vod)[0:4])

result = pd.DataFrame(result, columns = ['subsr', 'vod_id', 'real', 'predict'])
# result = result[['subsr', 'vod_id', 'predict']]
result

,subsr,vod_id,real,predict
0,59879000,0,None,0.129002
1,59879000,1,None,0.129262
2,59879000,3,None,0.184380
3,59879000,5,None,0.127621
4,59879000,6,None,0.336223
...,...,...,...,...
2409315,67164000,4708,None,0.145030
2409316,67164000,4710,None,0.145343
2409317,67164000,4711,None,0.145322
2409318,67164000,4712,None,0.145528


In [94]:
result = result[['subsr', 'vod_id', 'predict']]
result

,subsr,vod_id,predict
0,59879000,0,0.129002
1,59879000,1,0.129262
2,59879000,3,0.184380
3,59879000,5,0.127621
4,59879000,6,0.336223
...,...,...,...
2409315,67164000,4708,0.145030
2409316,67164000,4710,0.145343
2409317,67164000,4711,0.145322
2409318,67164000,4712,0.145528


In [95]:
result_1 = []
for user in user_id:
    result_1.append(result[result['subsr'] == user].sort_values(by = 'predict', ascending=False).vod_id[:30].tolist())

result_1 = pd.DataFrame(result_1)
result_1 = pd.concat([pd.DataFrame(user_id, columns = ['subsr']), result_1], axis = 1)
result_1

,subsr,0,1,2,3,4,5,6,7,8,...,20,21,22,23,24,25,26,27,28,29
0,59879000,2537,2288,313,2608,297,619,2057,293,143,...,533,247,4381,516,3927,2219,527,705,1025,231
1,59882000,2537,2288,2865,296,1166,2608,2057,1877,143,...,32,888,336,240,2556,2630,1233,780,256,472
2,59886000,2867,2865,2537,2057,2608,2630,610,2288,3175,...,1166,613,1267,240,780,818,401,7,2516,618
3,59890000,2556,2630,41,1156,759,116,1166,2537,888,...,2288,3177,432,2633,2865,2716,8,2866,619,2544
4,59892000,2537,2288,296,2865,1166,2057,1958,32,2608,...,3568,1772,1863,1227,9,1329,144,240,2585,2494
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
894,67149000,2537,2288,2057,2865,296,2608,1166,1958,32,...,3141,3183,868,3196,9,4223,581,1329,1214,2343
895,67154000,538,1433,1391,652,2866,1166,2288,296,1267,...,336,2537,2058,888,16,818,2321,1277,1885,48
896,67160000,2537,2288,2865,296,2057,1166,2608,1191,1958,...,3141,1863,137,3568,4223,1227,9,3705,538,1329
897,67161000,1215,888,2537,432,652,401,1433,335,333,...,16,143,2608,2630,2633,2865,1166,2866,2867,7


In [96]:
vod_predict = result_1.copy()
vod_predict.index = user_id
vod_predict = vod_predict.apply(lambda x : x[1:].tolist(), axis = 1)
vod_predict = vod_predict.reset_index()
vod_predict.columns = ['subsr', 'vod_id']
vod_predict

,subsr,vod_id
0,59879000,"[2537, 2288, 313, 2608, 297, 619, 2057, 293, 1..."
1,59882000,"[2537, 2288, 2865, 296, 1166, 2608, 2057, 1877..."
2,59886000,"[2867, 2865, 2537, 2057, 2608, 2630, 610, 2288..."
3,59890000,"[2556, 2630, 41, 1156, 759, 116, 1166, 2537, 8..."
4,59892000,"[2537, 2288, 296, 2865, 1166, 2057, 1958, 32, ..."
...,...,...
894,67149000,"[2537, 2288, 2057, 2865, 296, 2608, 1166, 1958..."
895,67154000,"[538, 1433, 1391, 652, 2866, 1166, 2288, 296, ..."
896,67160000,"[2537, 2288, 2865, 296, 2057, 1166, 2608, 1191..."
897,67161000,"[1215, 888, 2537, 432, 652, 401, 1433, 335, 33..."


In [97]:
# 추천 VOD가 영화인 경우, 본 적이 있다면 추천안함
TV_kids = vod_list[(vod_list['Category'] == 'TV프로그램') | (vod_list['Category'] == '키즈')].vod_id.unique().tolist()
movie = vod_list[vod_list['Category'] == '영화'].vod_id.unique().tolist()

vod_predict_1 = pd.DataFrame()
for user in user_id:
    li = []
    watched_list = con_score[con_score.subsr == user].vod_id.tolist()
    rec_list = vod_predict[vod_predict.subsr == user].vod_id.tolist()[0]
    for x in rec_list:
        if x not in watched_list:
            li.append(x)
        elif x in watched_list and x in TV_kids:
            li.append(x)
        elif x in watched_list and x in movie:
            continue

    vod_predict_1 = pd.concat([vod_predict_1, pd.DataFrame(li).T], axis= 0)

vod_predict_1.index = user_id
vod_predict_1 = vod_predict_1.apply(lambda x : x.tolist(), axis = 1)
vod_predict_1 = vod_predict_1.reset_index()
vod_predict_1.columns = ['subsr', 'vod_id']
vod_predict_1

,subsr,vod_id
0,59879000,"[2537.0, 2288.0, 313.0, 2608.0, 297.0, 619.0, ..."
1,59882000,"[2537.0, 2288.0, 2865.0, 296.0, 1166.0, 2608.0..."
2,59886000,"[2867.0, 2865.0, 2537.0, 2057.0, 2608.0, 2630...."
3,59890000,"[2556.0, 2630.0, 41.0, 1156.0, 759.0, 116.0, 1..."
4,59892000,"[2288.0, 296.0, 2865.0, 1166.0, 2057.0, 1958.0..."
...,...,...
894,67149000,"[2537.0, 2288.0, 2865.0, 296.0, 2608.0, 1166.0..."
895,67154000,"[538.0, 1433.0, 1391.0, 652.0, 2866.0, 1166.0,..."
896,67160000,"[2537.0, 2288.0, 2865.0, 296.0, 1166.0, 2608.0..."
897,67161000,"[1215.0, 888.0, 2537.0, 432.0, 652.0, 401.0, 1..."


In [98]:
testdata = pd.read_csv('/content/drive/MyDrive/LG Hellovision VOD 추천 프로젝트/모델/content_surprise/csv/watched_vod_10.csv', index_col=0)
testdata = testdata.groupby('subsr')['vod_id'].unique().reset_index()
testdata

,subsr,vod_id
0,59900000,"[1278, 1885]"
1,59930000,[2098]
2,59933000,[296]
3,60050000,"[200, 2415, 2869, 2546, 213, 2291, 2610, 4880]"
4,60067000,"[1785, 887, 1347, 4773, 4782, 4759, 992, 3806,..."
...,...,...
215,67133000,[296]
216,67164000,"[4956, 1434, 4995, 2058]"
217,67170000,"[3464, 948, 3900, 675]"
218,67202000,[1028]


In [ ]:
# precision@k
def precision_k(testdata, recommended_data, K = 10):
    merge_df = pd.merge(testdata, recommended_data, on='subsr', how = 'left', suffixes=('_actual', '_rec'))
    merge_df = merge_df.dropna()
    precision_value = 0
    for i in range(merge_df.shape[0]):
        temp = len(set(merge_df.iloc[i,1]).intersection(set(merge_df.iloc[i, 2][:K]))) / K
        precision_value += temp

    return precision_value / merge_df.shape[0]

precision_k(testdata, vod_predict_1)

0.026153846153846153

### KNNBaseline - item-based similarity

### SVD, SVDpp

In [85]:
from surprise.model_selection import GridSearchCV

param_grid = {'n_epochs': np.arange(5, 100, 5), 'lr_all': [0.002, 0.005],
              'reg_all': [0.04, 0.06], 'random_state' : [42]}
              # lr_all – The learning rate for all parameters. Default is 0.005.
              # reg_all – The regularization term for all parameters. Default is 0.02.

gs = GridSearchCV(SVD, param_grid, measures=['mae'], cv=5)

reader = Reader(rating_scale=(0,1))
train_set = Dataset.load_from_df(con_score[['subsr', 'vod_id', 'click_score']], reader)

gs.fit(train_set)
gs.best_params

KeyboardInterrupt: ignored

In [86]:
# n_epochs=95, lr_all=0.005, reg_all=0.06, random_state=42 -> 0.045
svd = SVD(n_epochs=200, lr_all=0.003, reg_all=0.06, random_state=42)
svd.fit(trainset)

In [87]:
user_id = sorted(con_score.subsr.unique())
vod_id = sorted(con_score.vod_id.unique())

result = []
for user in user_id:
    for vod in vod_id:
        result.append(svd.predict(user, vod)[0:4])

result = pd.DataFrame(result, columns = ['subsr', 'vod_id', 'real', 'predict'])
result = result[['subsr', 'vod_id', 'predict']]
result

,subsr,vod_id,predict
0,59879000,0,0.267358
1,59879000,1,0.101047
2,59879000,3,0.223533
3,59879000,5,0.190077
4,59879000,6,0.203941
...,...,...,...
2409315,67164000,4708,0.283825
2409316,67164000,4710,0.401331
2409317,67164000,4711,0.252782
2409318,67164000,4712,0.124780


In [88]:
result_1 = []
for user in user_id:
    result_1.append(result[result['subsr'] == user].sort_values(by = 'predict', ascending=False).vod_id[:30].tolist())

result_1 = pd.DataFrame(result_1)
result_1 = pd.concat([pd.DataFrame(user_id, columns = ['subsr']), result_1], axis = 1)
result_1

,subsr,0,1,2,3,4,5,6,7,8,...,20,21,22,23,24,25,26,27,28,29
0,59879000,1243,3838,1863,979,784,789,1191,2537,498,...,1599,1214,1772,1130,2838,2057,510,1921,3196,3954
1,59882000,3183,296,789,2307,1243,2537,112,3285,2288,...,2865,240,137,710,4223,581,1027,2148,177,738
2,59886000,979,137,2576,112,1329,3183,1958,308,3033,...,4118,3776,2537,2302,7,2556,1233,3001,2667,1877
3,59890000,2716,137,784,1329,3183,754,3705,2226,3779,...,868,448,1749,1877,1050,4121,4146,2494,2221,296
4,59892000,1958,868,2867,137,2537,728,1233,144,1599,...,2560,3705,3184,445,3029,3196,2949,538,2865,240
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
894,67149000,1191,3141,868,1863,3196,1243,296,979,1227,...,2949,112,3891,1599,137,2290,3272,4650,2288,1709
895,67154000,476,32,1401,847,4121,3838,1191,240,1877,...,296,2537,1107,2875,549,509,9,1243,3705,316
896,67160000,112,4114,1214,2718,32,1227,821,2343,4446,...,316,1362,296,3768,4138,610,2109,882,2537,1786
897,67161000,1191,868,3824,4612,458,4101,4223,2226,1575,...,3196,2949,2537,320,2288,433,4469,2024,2865,80


In [89]:
vod_predict = result_1.copy()
vod_predict.index = user_id
vod_predict = vod_predict.apply(lambda x : x[1:].tolist(), axis = 1)
vod_predict = vod_predict.reset_index()
vod_predict.columns = ['subsr', 'vod_id']
vod_predict

,subsr,vod_id
0,59879000,"[1243, 3838, 1863, 979, 784, 789, 1191, 2537, ..."
1,59882000,"[3183, 296, 789, 2307, 1243, 2537, 112, 3285, ..."
2,59886000,"[979, 137, 2576, 112, 1329, 3183, 1958, 308, 3..."
3,59890000,"[2716, 137, 784, 1329, 3183, 754, 3705, 2226, ..."
4,59892000,"[1958, 868, 2867, 137, 2537, 728, 1233, 144, 1..."
...,...,...
894,67149000,"[1191, 3141, 868, 1863, 3196, 1243, 296, 979, ..."
895,67154000,"[476, 32, 1401, 847, 4121, 3838, 1191, 240, 18..."
896,67160000,"[112, 4114, 1214, 2718, 32, 1227, 821, 2343, 4..."
897,67161000,"[1191, 868, 3824, 4612, 458, 4101, 4223, 2226,..."


In [90]:
# 추천 VOD가 영화인 경우, 본 적이 있다면 추천안함
# vod_list = pd.read_csv('../data/vod_list_add10.csv', index_col=0)
TV_kids = vod_list[(vod_list['Category'] == 'TV프로그램') | (vod_list['Category'] == '키즈')].vod_id.unique().tolist()
movie = vod_list[vod_list['Category'] == '영화'].vod_id.unique().tolist()

vod_predict_1 = pd.DataFrame()
for user in user_id:
    li = []
    watched_list = con_score[con_score.subsr == user].vod_id.tolist()
    rec_list = vod_predict[vod_predict.subsr == user].vod_id.tolist()[0]
    for x in rec_list:
        if x not in watched_list:
            li.append(x)
        elif x in watched_list and x in TV_kids:
            li.append(x)
        elif x in watched_list and x in movie:
            continue

    vod_predict_1 = pd.concat([vod_predict_1, pd.DataFrame(li).T], axis= 0)

vod_predict_1.index = user_id
vod_predict_1 = vod_predict_1.apply(lambda x : x.tolist(), axis = 1)
vod_predict_1 = vod_predict_1.reset_index()
vod_predict_1.columns = ['subsr', 'vod_id']
vod_predict_1

,subsr,vod_id
0,59879000,"[1243.0, 3838.0, 1863.0, 979.0, 784.0, 789.0, ..."
1,59882000,"[3183.0, 296.0, 789.0, 2307.0, 1243.0, 2537.0,..."
2,59886000,"[979.0, 137.0, 2576.0, 112.0, 1329.0, 3183.0, ..."
3,59890000,"[137.0, 784.0, 1329.0, 3183.0, 754.0, 3705.0, ..."
4,59892000,"[1958.0, 868.0, 2867.0, 137.0, 728.0, 1233.0, ..."
...,...,...
894,67149000,"[1191.0, 3141.0, 868.0, 1863.0, 3196.0, 1243.0..."
895,67154000,"[476.0, 32.0, 1401.0, 847.0, 4121.0, 3838.0, 1..."
896,67160000,"[112.0, 4114.0, 1214.0, 2718.0, 32.0, 1227.0, ..."
897,67161000,"[1191.0, 868.0, 4612.0, 458.0, 4101.0, 4223.0,..."


In [ ]:
testdata = pd.read_csv('../data/watched_vod_10.csv', index_col=0)
testdata = testdata.groupby('subsr')['vod_id'].unique().reset_index()
testdata

,subsr,vod_id
0,59900000,"[1278, 1885]"
1,59930000,[2098]
2,59933000,[296]
3,60050000,"[200, 2415, 2869, 2546, 213, 2291, 2610, 4880]"
4,60067000,"[1785, 887, 1347, 4773, 4782, 4759, 992, 3806,..."
...,...,...
215,67133000,[296]
216,67164000,"[4956, 1434, 4995, 2058]"
217,67170000,"[3464, 948, 3900, 675]"
218,67202000,[1028]


In [91]:
# precision@k
def precision_k(testdata, recommended_data, K = 10):
    merge_df = pd.merge(testdata, recommended_data, on='subsr', how = 'left', suffixes=('_actual', '_rec'))
    merge_df = merge_df.dropna()
    precision_value = 0
    for i in range(merge_df.shape[0]):
        temp = len(set(merge_df.iloc[i,1]).intersection(set(merge_df.iloc[i, 2][:K]))) / K
        precision_value += temp

    return precision_value / merge_df.shape[0]

precision_k(testdata, vod_predict)

0.010769230769230772